In [2]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, BatchNormalization, Embedding, LeakyReLU, Dropout
from keras.activations import relu, sigmoid

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [7]:
dataset = pd.read_csv('/content/drive/MyDrive/AI/DL/Churn/Churn_Modelling.csv')

In [8]:
X = dataset.iloc[:,3:13]
y = dataset.iloc[:,13]

In [9]:
geography = pd.get_dummies(X['Geography'],drop_first=True)
gender=pd.get_dummies(X['Gender'],drop_first=True)

In [10]:
X = pd.concat([X,geography,gender],axis=1)
X = X.drop(['Geography','Gender'],axis=1)

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [13]:
def create_model(layers, activation):
  model = Sequential()
  for i, nodes in enumerate(layers):
    if i==0:
      model.add(Dense(nodes,input_dim=X_train.shape[1]))
      model.add(Activation(relu))
      model.add(Dropout(0.3))
    else:
      model.add(Dense(nodes))
      model.add(Activation(activation))
      model.add(Dropout(0.3))
  model.add(Dense(units=1,kernel_initializer='glorot_uniform',activation='sigmoid'))
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])
  return model

In [15]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [16]:
model = KerasClassifier(build_fn=create_model,verbose=0)

In [23]:
layers = [(20,),(40,20),(45,30,15)]
activations = ['sigmoid', 'relu']

In [24]:
param_grid = dict(layers=layers,activation=activations,batch_size=[128,256],epochs=[30])

In [25]:
grid = GridSearchCV(estimator=model,param_grid=param_grid,cv=5)

In [26]:
grid_result = grid.fit(X_train,y_train)

In [29]:
grid_result.best_params_, grid_result.best_score_

({'activation': 'sigmoid',
  'batch_size': 128,
  'epochs': 30,
  'layers': (40, 20)},
 0.7959999918937684)

In [30]:
y_pred = grid.predict(X_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [31]:
y_pred = (y_pred > 0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [32]:
from sklearn.metrics import confusion_matrix,accuracy_score

In [33]:
confusion_matrix(y_pred,y_test)

array([[1595,  405],
       [   0,    0]])

In [34]:
accuracy_score(y_pred,y_test)

0.7975